In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# 核心代码

In [2]:
import argparse
import glob
import logging
import os
import json
import time
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from callback.optimizater.adamw import AdamW
from callback.lr_scheduler import get_linear_schedule_with_warmup
from callback.progressbar import ProgressBar
from callback.adversarial import FGM
from tools.common import seed_everything, json_to_text
from tools.common import init_logger, logger

from models.transformers import WEIGHTS_NAME, BertConfig, AlbertConfig
from models.bert_for_ner import BertSpanForNer
from models.albert_for_ner import AlbertSpanForNer
from processors.utils_ner import CNerTokenizer
from processors.ner_span_for_heats import convert_examples_to_features
from processors.ner_span_for_heats import ner_processors as processors
from processors.ner_span_for_heats import collate_fn
from metrics.ner_metrics import SpanEntityScore
from processors.utils_ner import bert_extract_item

import collections

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
MODEL_CLASSES = {
    ## bert ernie bert_wwm bert_wwwm_ext
    'bert': (BertConfig, BertSpanForNer, CNerTokenizer),
    'albert': (AlbertConfig, AlbertSpanForNer, CNerTokenizer)
}

def index2token(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip('\n')
        vocab[index] = token
    return vocab
index2token=index2token('/home/DUTIR/suncong/NER_2021_github/biobert_v1.1_v2.5.1/vocab.txt')

def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size,
                                  collate_fn=collate_fn)
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        
#     # Prepare optimizer and schedule (linear warmup and decay)
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {
#             "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": args.weight_decay,
#         },
#         {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
#     ]
#     args.warmup_steps = int(t_total * args.warmup_proportion)
#     optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
#     )
        
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    bert_parameters = model.bert.named_parameters()
    start_parameters = model.start_fc.named_parameters()
    end_parameters = model.end_fc.named_parameters()
    optimizer_grouped_parameters = [
        {"params": [p for n, p in bert_parameters if not any(nd in n for nd in no_decay)],
         "weight_decay": args.weight_decay, 'lr': args.learning_rate},
        {"params": [p for n, p in bert_parameters if any(nd in n for nd in no_decay)], "weight_decay": 0.0
            , 'lr': args.learning_rate},

        {"params": [p for n, p in start_parameters if not any(nd in n for nd in no_decay)],
         "weight_decay": args.weight_decay, 'lr': 0.001},
        {"params": [p for n, p in start_parameters if any(nd in n for nd in no_decay)], "weight_decay": 0.0
            , 'lr': 0.001},

        {"params": [p for n, p in end_parameters if not any(nd in n for nd in no_decay)],
         "weight_decay": args.weight_decay, 'lr': 0.001},
        {"params": [p for n, p in end_parameters if any(nd in n for nd in no_decay)], "weight_decay": 0.0
            , 'lr': 0.001},
    ]
    # optimizer_grouped_parameters = [
    #     {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    #      "weight_decay": args.weight_decay, },
    #     {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    # ]
    args.warmup_steps = int(t_total * args.warmup_proportion)
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                num_training_steps=t_total)
    print('t_total:',t_total)
    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)
    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)
    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank],
                                                          output_device=args.local_rank,
                                                          find_unused_parameters=True)
    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                args.train_batch_size
                * args.gradient_accumulation_steps
                * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
                )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path) and "checkpoint" in args.model_name_or_path:
        # set global_step to gobal_step of last saved checkpoint from model path
        global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)
        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
    tr_loss, logging_loss = 0.0, 0.0
    best_f1 = 0.0 
    
    if args.do_adv:
        fgm = FGM(model, emb_name=args.adv_name, epsilon=args.adv_epsilon)
    model.zero_grad()
    seed_everything(args.seed)  # Added here for reproductibility (even between python 2 and 3)
    for _ in range(int(args.num_train_epochs)):
        print('Epoch:',_+1)
        pbar = ProgressBar(n_total=len(train_dataloader), desc='Training')
        for step, batch in enumerate(train_dataloader):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {"input_ids": batch[0], "attention_mask": batch[1],
                      "start_positions": batch[3], "end_positions": batch[4]}
            if args.model_type != "distilbert":
                # XLM and RoBERTa don"t use segment_ids
                inputs["token_type_ids"] = (batch[2] if args.model_type in ["bert", "xlnet"] else None)
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            if args.do_adv:
                fgm.attack()
                loss_adv = model(**inputs)[0]
                if args.n_gpu > 1:
                    loss_adv = loss_adv.mean()
                loss_adv.backward()
                fgm.restore()
            pbar(step, {'loss': loss.item()})
            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1
                
                args.logging_steps = int(t_total/args.num_train_epochs)
#                 if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                if args.local_rank in [-1, 0] and global_step >= 1*args.logging_steps and global_step % args.logging_steps == 0:
                    # Log metrics
                    logger.info("\n")
                    if args.local_rank == -1:
                        # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        
                        if results['f1'] > best_f1:
                            
#                             output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                            output_dir = os.path.join(args.output_dir)
                            if not os.path.exists(output_dir):
                                os.makedirs(output_dir)
                            model_to_save = (
                                model.module if hasattr(model, "module") else model
                            )  # Take care of distributed/parallel training
                            model_to_save.save_pretrained(output_dir)
                            torch.save(args, os.path.join(output_dir, "training_args.bin"))
                            tokenizer.save_vocabulary(output_dir)
                            logger.info("Saving model checkpoint to %s", output_dir)
                            torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                            torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                            logger.info("Saving optimizer and scheduler states to %s", output_dir)     
                            
                            best_f1 = results['f1']
                            
                            
                    print('training loss one epoch:',(tr_loss - logging_loss))
                    logging_loss = tr_loss                            
                                           
                        
                        
                        
                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    tokenizer.save_vocabulary(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)
                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)
        logger.info("\n")
        if 'cuda' in str(args.device):
            torch.cuda.empty_cache()
    return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, prefix=""):
    metric = SpanEntityScore(args.id2label)
    eval_output_dir = args.output_dir
    if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir)
    eval_features = load_and_cache_examples(args, args.task_name, tokenizer, data_type='dev')
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Eval!
    logger.info("***** Running evaluation %s *****", prefix)
    logger.info("  Num examples = %d", len(eval_features))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    pbar = ProgressBar(n_total=len(eval_features), desc="Evaluating")
    for step, f in enumerate(eval_features):
        input_lens = f.input_len
        input_ids = torch.tensor([f.input_ids[:input_lens]], dtype=torch.long).to(args.device)
        input_mask = torch.tensor([f.input_mask[:input_lens]], dtype=torch.long).to(args.device)
        segment_ids = torch.tensor([f.segment_ids[:input_lens]], dtype=torch.long).to(args.device)
        start_ids = torch.tensor([f.start_ids[:input_lens]], dtype=torch.long).to(args.device)
        end_ids = torch.tensor([f.end_ids[:input_lens]], dtype=torch.long).to(args.device)
        subjects = f.subjects
        model.eval()
        with torch.no_grad():
            inputs = {"input_ids": input_ids, "attention_mask": input_mask,
                      "start_positions": start_ids, "end_positions": end_ids}
            if args.model_type != "distilbert":
                # XLM and RoBERTa don"t use segment_ids
                inputs["token_type_ids"] = (segment_ids if args.model_type in ["bert", "xlnet"] else None)
            outputs = model(**inputs)
#         print('input_mask:',input_mask)
#         print('segment_ids:',segment_ids)
        input_lens_new = torch.sum(input_mask.view(-1)-segment_ids.view(-1))
#         print(input_lens_new)
        
        tmp_eval_loss, start_logits, end_logits = outputs[:3]
        R_final = bert_extract_item(start_logits, end_logits, input_lens_new.cpu())
        
#         R_final=[]
#         if len(R) < 2:
#             R_final=R
#         else:
#             for i in range(len(R)-1):
#                 if R[i][2] < R[i+1][1]:
#                     R_final.append(R[i])
#             R_final.append(R[len(R)-1])
        
        T = subjects
        
        metric.update(true_subject=T, pred_subject=R_final)
        if args.n_gpu > 1:
            tmp_eval_loss = tmp_eval_loss.mean()  # mean() to average on multi-gpu parallel evaluating
        eval_loss += tmp_eval_loss.item()
        nb_eval_steps += 1
        pbar(step)
    logger.info("\n")
    eval_loss = eval_loss / nb_eval_steps
    eval_info, entity_info = metric.result()
    results = {f'{key}': value for key, value in eval_info.items()}
    results['loss'] = eval_loss
    logger.info("***** Eval results %s *****", prefix)
    info = "-".join([f' {key}: {value:.4f} ' for key, value in results.items()])
    logger.info(info)
    logger.info("***** Entity results %s *****", prefix)
    for key in sorted(entity_info.keys()):
        logger.info("******* %s results ********" % key)
        info = "-".join([f' {key}: {value:.4f} ' for key, value in entity_info[key].items()])
        logger.info(info)
    return results


def predict(args, model, tokenizer, prefix=""):
    metric = SpanEntityScore(args.id2label)
    pred_output_dir = args.output_dir
    if not os.path.exists(pred_output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(pred_output_dir)
    test_features = load_and_cache_examples(args, args.task_name, tokenizer, data_type='test')
#     args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)

    # Eval!
    logger.info("***** Running prediction %s *****", prefix)
    logger.info("  Num examples = %d", len(test_features))
    logger.info("  Batch size = %d", 1)
    
    test_loss = 0.0
    nb_test_steps = 0
    pbar = ProgressBar(n_total=len(test_features), desc="Predicting")

    results = []
    output_predict_file = os.path.join(pred_output_dir, prefix, "test_predict.json")

#     all_attentions = []
    for step, f in enumerate(test_features):
        input_lens = f.input_len
        input_ids = torch.tensor([f.input_ids[:input_lens]], dtype=torch.long).to(args.device)
        input_mask = torch.tensor([f.input_mask[:input_lens]], dtype=torch.long).to(args.device)
        segment_ids = torch.tensor([f.segment_ids[:input_lens]], dtype=torch.long).to(args.device)
        start_ids = torch.tensor([f.start_ids[:input_lens]], dtype=torch.long).to(args.device)
        end_ids = torch.tensor([f.end_ids[:input_lens]], dtype=torch.long).to(args.device)
        subjects = f.subjects
        
        model.eval()

        with torch.no_grad():
            inputs = {"input_ids": input_ids, "attention_mask": input_mask,
                      "start_positions": start_ids, "end_positions": end_ids}
            if args.model_type != "distilbert":
                # XLM and RoBERTa don"t use segment_ids
                inputs["token_type_ids"] = (segment_ids if args.model_type in ["bert", "xlnet"] else None)
            outputs = model(**inputs)
        
        
        input_lens_new = torch.sum(input_mask.view(-1)-segment_ids.view(-1))
        tmp_test_loss, start_logits, end_logits = outputs[:3]

#         all_attentions.append(outputs[4])
        
        R_final = bert_extract_item(start_logits, end_logits, input_lens_new.cpu())
        
#         R_final=[]
#         if len(R) < 2:
#             R_final=R
#         else:
#             for i in range(len(R)-1):
#                 if R[i][2] < R[i+1][1]:
#                     R_final.append(R[i])
#             R_final.append(R[len(R)-1])
        
        T = subjects
        metric.update(true_subject=T, pred_subject=R_final)

        test_loss += tmp_test_loss.item()
        nb_test_steps += 1
        pbar(step)        
        
        
        if R_final:
            label_entities = [[args.id2label[x[0]], x[1], x[2]] for x in R_final]
        else:
            label_entities = []
        
        
        
        sequence_ids = input_ids.squeeze(0).cpu().numpy().tolist()

        tokens = []
        for token_id in sequence_ids:
            tokens.append(index2token[token_id])
        
        length = len(tokens[1:-1])
        tag_seq = length*['O']
        for e in label_entities:
            tag_seq[e[1]]='B-BioNE'
            count=1
            while e[1]+count<=e[2]:
                tag_seq[e[1]+count] = 'I-BioNE'
                count +=1        
        
        json_d = {}
        json_d['id'] = step
        json_d['token_seq'] = tokens[1:-1]
        json_d['tag_seq'] = tag_seq
        json_d['entities'] = label_entities
        results.append(json_d)
        pbar(step)
    logger.info("\n")
    with open(output_predict_file, "w") as writer:
        for record in results:
            writer.write(json.dumps(record) + '\n')
            
            
    logger.info("\n")
    test_loss = test_loss / nb_test_steps
    test_info, test_entity_info = metric.result()
    test_results = {f'{key}': value for key, value in test_info.items()}
    test_results['loss'] = test_loss
    logger.info("***** Test results %s *****", prefix)
    info = "-".join([f' {key}: {value:.4f} ' for key, value in test_results.items()])
    logger.info(info)
    logger.info("***** Test entity results %s *****", prefix)
    for key in sorted(test_entity_info.keys()):
        logger.info("******* %s results ********" % key)
        info = "-".join([f' {key}: {value:.4f} ' for key, value in test_entity_info[key].items()])
        logger.info(info)
        
#     return all_attentions


def load_and_cache_examples(args, task, tokenizer, data_type='train'):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
    processor = processors[task]()
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'cached_span-{}_{}_{}_{}'.format(
        data_type,
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.train_max_seq_length if data_type == 'train' else args.eval_max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if data_type == 'train':
            examples = processor.get_train_examples(args.data_dir)
        elif data_type == 'dev':
            examples = processor.get_dev_examples(args.data_dir)
        else:
            examples = processor.get_test_examples(args.data_dir)
        features = convert_examples_to_features(examples=examples,
                                                tokenizer=tokenizer,
                                                label_list=label_list,
                                                max_seq_length=args.train_max_seq_length if data_type == 'train' \
                                                    else args.eval_max_seq_length,
                                                cls_token_at_end=bool(args.model_type in ["xlnet"]),
                                                pad_on_left=bool(args.model_type in ['xlnet']),
                                                cls_token=tokenizer.cls_token,
                                                cls_token_segment_id=2 if args.model_type in ["xlnet"] else 0,
                                                sep_token=tokenizer.sep_token,
                                                # pad on the left for xlnet
                                                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0,
                                                )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
    # Convert to Tensors and build dataset
    if data_type == 'dev' or data_type == 'test':
        return features
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_start_ids = torch.tensor([f.start_ids for f in features], dtype=torch.long)
    all_end_ids = torch.tensor([f.end_ids for f in features], dtype=torch.long)
    all_input_lens = torch.tensor([f.input_len for f in features], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_start_ids, all_end_ids, all_input_lens)
    return dataset




import argparse

# def get_argparse():
parser = argparse.ArgumentParser()
# Required parameters
parser.add_argument("--task_name", default=None, type=str, required=True,
                    help="The name of the task to train selected in the list: ")
parser.add_argument("--data_dir", default=None, type=str, required=True,
                    help="The input data dir. Should contain the training files for the CoNLL-2003 NER task.", )
parser.add_argument("--model_type", default=None, type=str, required=True,
                    help="Model type selected in the list: ")
parser.add_argument("--model_name_or_path", default=None, type=str, required=True,
                    help="Path to pre-trained model or shortcut name selected in the list: " )
parser.add_argument("--output_dir", default=None, type=str, required=True,
                    help="The output directory where the model predictions and checkpoints will be written.", )

# Other parameters
parser.add_argument('--markup', default='bio', type=str,
                    choices=['bios', 'bio'])
parser.add_argument('--loss_type', default='ce', type=str,
                    choices=['lsr', 'focal', 'ce'])
parser.add_argument("--config_name", default="", type=str,
                    help="Pretrained config name or path if not the same as model_name")
parser.add_argument("--tokenizer_name", default="", type=str,
                    help="Pretrained tokenizer name or path if not the same as model_name", )
parser.add_argument("--cache_dir", default="", type=str,
                    help="Where do you want to store the pre-trained models downloaded from s3", )
parser.add_argument("--train_max_seq_length", default=128, type=int,
                    help="The maximum total input sequence length after tokenization. Sequences longer "
                         "than this will be truncated, sequences shorter will be padded.", )
parser.add_argument("--eval_max_seq_length", default=512, type=int,
                    help="The maximum total input sequence length after tokenization. Sequences longer "
                         "than this will be truncated, sequences shorter will be padded.", )
parser.add_argument("--do_train", action="store_true",
                    help="Whether to run training.")
parser.add_argument("--do_eval", action="store_true",
                    help="Whether to run eval on the dev set.")
parser.add_argument("--do_predict", action="store_true",
                    help="Whether to run predictions on the test set.")
parser.add_argument("--evaluate_during_training", action="store_true",
                    help="Whether to run evaluation during training at each logging step.", )
parser.add_argument("--do_lower_case", action="store_true",
                    help="Set this flag if you are using an uncased model.")
# adversarial training
parser.add_argument("--do_adv", action="store_true",
                    help="Whether to adversarial training.")
parser.add_argument('--adv_epsilon', default=1.0, type=float,
                    help="Epsilon for adversarial.")
parser.add_argument('--adv_name', default='word_embeddings', type=str,
                    help="name for adversarial layer.")

parser.add_argument("--per_gpu_train_batch_size", default=8, type=int,
                    help="Batch size per GPU/CPU for training.")
parser.add_argument("--per_gpu_eval_batch_size", default=8, type=int,
                    help="Batch size per GPU/CPU for evaluation.")
parser.add_argument("--gradient_accumulation_steps", type=int, default=1,
                    help="Number of updates steps to accumulate before performing a backward/update pass.", )
parser.add_argument("--learning_rate", default=5e-5, type=float,
                    help="The initial learning rate for Adam.")
parser.add_argument("--crf_learning_rate", default=5e-5, type=float,
                    help="The initial learning rate for crf and linear layer.")
parser.add_argument("--weight_decay", default=0.01, type=float,
                    help="Weight decay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                    help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                    help="Max gradient norm.")
parser.add_argument("--num_train_epochs", default=3.0, type=float,
                    help="Total number of training epochs to perform.")
parser.add_argument("--max_steps", default=-1, type=int,
                    help="If > 0: set total number of training steps to perform. Override num_train_epochs.", )

parser.add_argument("--warmup_proportion", default=0.1, type=float,
                    help="Proportion of training to perform linear learning rate warmup for,E.g., 0.1 = 10% of training.")
parser.add_argument("--logging_steps", type=int, default=50,
                    help="Log every X updates steps.")
parser.add_argument("--save_steps", type=int, default=50, help="Save checkpoint every X updates steps.")
parser.add_argument("--eval_all_checkpoints", action="store_true",
                    help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number", )
parser.add_argument("--predict_checkpoints",type=int, default=0,
                    help="predict checkpoints starting with the same prefix as model_name ending and ending with step number")
parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--overwrite_output_dir", action="store_true",
                    help="Overwrite the content of the output directory")
parser.add_argument("--overwrite_cache", action="store_true",
                    help="Overwrite the cached training and evaluation sets")
parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
parser.add_argument("--fp16", action="store_true",
                    help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit", )
parser.add_argument("--fp16_opt_level", type=str, default="O1",
                    help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                         "See details at https://nvidia.github.io/apex/amp.html", )
parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
#     return parser


bert_model_url = '/home/DUTIR/suncong/NER_2021_github/biobert_v1.1_v2.5.1'
data_dir_url = '/home/DUTIR/suncong/NER_2021_github/datasets/BC4CHEMD/mrc-ner-top3-chem'
output_dir_url = '/home/DUTIR/suncong/NER_2021_github/output/BC4CHEMD_MRC_seed_17_top3_epochs_1_'

args = parser.parse_args(['--data_dir',data_dir_url,'--model_type','bert',
                          '--model_name_or_path',bert_model_url, '--output_dir',output_dir_url,'--evaluate_during_training',
                          '--train_max_seq_length','512', '--eval_max_seq_length','512','--num_train_epochs','1',
                          '--per_gpu_train_batch_size','8', '--task_name', 'bner', '--loss_type', 'ce', '--logging_steps','100000',
                          '--per_gpu_eval_batch_size','8','--save_steps','100000','--seed','17','--learning_rate','3e-5',
#                           '--do_train',
                          '--do_eval','--do_predict', '--overwrite_output_dir'])

print(args)


# def main():
# args = get_argparse().parse_args()
if not os.path.exists(args.output_dir):
    os.mkdir(args.output_dir)
args.output_dir = args.output_dir + '{}'.format(args.model_type)
if not os.path.exists(args.output_dir):
    os.mkdir(args.output_dir)
time_ = time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime())
init_logger(log_file=args.output_dir + f'/{args.model_type}-{args.task_name}-{time_}.log')
if os.path.exists(args.output_dir) and os.listdir(
        args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError(
        "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
            args.output_dir))
# Setup distant debugging if needed
if args.server_ip and args.server_port:
    # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
    import ptvsd
    print("Waiting for debugger attach")
    ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
    ptvsd.wait_for_attach()
# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device
logger.warning("Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
               args.local_rank, device, args.n_gpu, bool(args.local_rank != -1), args.fp16, )
# Set seed
seed_everything(args.seed)
# Prepare NER task
args.task_name = args.task_name.lower()
if args.task_name not in processors:
    raise ValueError("Task not found: %s" % (args.task_name))
processor = processors[args.task_name]()
label_list = processor.get_labels()
args.id2label = {i: label for i, label in enumerate(label_list)}
args.label2id = {label: i for i, label in enumerate(label_list)}
num_labels = len(label_list)

# Load pretrained model and tokenizer
if args.local_rank not in [-1, 0]:
    torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
args.model_type = args.model_type.lower()
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
# print('tokenizer_class:',tokenizer_class)
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                      num_labels=num_labels,
                                      loss_type=args.loss_type,
                                      cache_dir=args.cache_dir if args.cache_dir else None,
                                      soft_label=True)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
                                            do_lower_case=args.do_lower_case,
                                            cache_dir=args.cache_dir if args.cache_dir else None, )
model = model_class.from_pretrained(args.model_name_or_path,
                                    from_tf=bool(".ckpt" in args.model_name_or_path),
                                    config=config)
if args.local_rank == 0:
    torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

model.to(args.device)
logger.info("Training/evaluation parameters %s", args)
# Training
if args.do_train:
    train_dataset = load_and_cache_examples(args, args.task_name, tokenizer, data_type='train')
    global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

results = {}
if args.do_eval and args.local_rank in [-1, 0]:
    tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
    checkpoints = [args.output_dir]
    if args.eval_all_checkpoints:
        checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
        )
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
        prefix = checkpoint.split('/')[-1] if checkpoint.find('checkpoint') != -1 else ""
        model = model_class.from_pretrained(checkpoint)
        model.to(args.device)
        result = evaluate(args, model, tokenizer, prefix=prefix)
        if global_step:
            result = {"{}_{}".format(global_step, k): v for k, v in result.items()}
        results.update(result)
    output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        for key in sorted(results.keys()):
            writer.write("{} = {}\n".format(key, str(results[key])))

if args.do_predict and args.local_rank in [-1, 0]:
    tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
    checkpoints = [args.output_dir]
    if args.predict_checkpoints > 0:
        checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        checkpoints = [x for x in checkpoints if x.split('-')[-1] == str(args.predict_checkpoints)]
    logger.info("Predict the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        prefix = checkpoint.split('/')[-1] if checkpoint.find('checkpoint') != -1 else ""
        model = model_class.from_pretrained(checkpoint)
        model.to(args.device)
        all_attentions = predict(args, model, tokenizer, prefix=prefix)


# if __name__ == "__main__":
#     main()

09/25/2021 20:43:42 - WARNING - root -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/25/2021 20:43:42 - INFO - models.transformers.configuration_utils -   loading configuration file /home/DUTIR/suncong/NER_2021_github/biobert_v1.1_v2.5.1/config.json
09/25/2021 20:43:42 - INFO - models.transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "loss_type": "ce",
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": "true",
  "output_hidden_states": "true",
  "output_past": true,
  "pruned_heads": {},
  "soft_label": true,
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

09/25/2021 20:43:42 -

Namespace(adam_epsilon=1e-08, adv_epsilon=1.0, adv_name='word_embeddings', cache_dir='', config_name='', crf_learning_rate=5e-05, data_dir='/home/DUTIR/suncong/NER_2021_github/datasets/BC4CHEMD/mrc-ner-top3-chem', do_adv=False, do_eval=True, do_lower_case=False, do_predict=True, do_train=False, eval_all_checkpoints=False, eval_max_seq_length=512, evaluate_during_training=True, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, learning_rate=3e-05, local_rank=-1, logging_steps=100000, loss_type='ce', markup='bio', max_grad_norm=1.0, max_steps=-1, model_name_or_path='/home/DUTIR/suncong/NER_2021_github/biobert_v1.1_v2.5.1', model_type='bert', no_cuda=False, num_train_epochs=1.0, output_dir='/home/DUTIR/suncong/NER_2021_github/output/BC4CHEMD_MRC_seed_17_top3_epochs_1_', overwrite_cache=False, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, predict_checkpoints=0, save_steps=100000, seed=17, server_ip='', server_port='', task_name='bner', toke

09/25/2021 20:43:47 - INFO - models.transformers.modeling_utils -   Weights of BertSpanForNer not initialized from pretrained model: ['start_fc.dense.weight', 'start_fc.dense.bias', 'end_fc.dense_0.weight', 'end_fc.dense_0.bias', 'end_fc.LayerNorm.weight', 'end_fc.LayerNorm.bias', 'end_fc.dense_1.weight', 'end_fc.dense_1.bias']
09/25/2021 20:43:47 - INFO - models.transformers.modeling_utils -   Weights from pretrained model not used in BertSpanForNer: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
09/25/2021 20:43:50 - INFO - root -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, adv_epsilon=1.0, adv_name='word_embeddings', cache_dir='', config_name='', crf_learning_rate=5e-05, data_dir='/home/DUTIR/

[Evaluating] 6310/6318 [============================>.] - ETA: 0ss0

09/25/2021 20:46:27 - INFO - root -   

09/25/2021 20:46:27 - INFO - root -   right : 5377
09/25/2021 20:46:27 - INFO - root -   found : 5773
09/25/2021 20:46:27 - INFO - root -   origin : 5788
09/25/2021 20:46:27 - INFO - root -   right : 5377
09/25/2021 20:46:27 - INFO - root -   found : 5773
09/25/2021 20:46:27 - INFO - root -   origin : 5788
09/25/2021 20:46:27 - INFO - root -   ***** Eval results  *****
09/25/2021 20:46:27 - INFO - root -    precision: 0.9314 - recall: 0.9290 - f1: 0.9302 - loss: 0.0060 
09/25/2021 20:46:27 - INFO - root -   ***** Entity results  *****
09/25/2021 20:46:27 - INFO - root -   ******* BioNE results ********
09/25/2021 20:46:27 - INFO - root -    precision: 0.9314 - recall: 0.9290 - f1: 0.9302 


...

09/25/2021 20:46:27 - INFO - models.transformers.tokenization_utils -   Model name '/home/DUTIR/suncong/NER_2021_github/output/BC4CHEMD_MRC_seed_17_top3_epochs_1_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased). Assuming '/home/DUTIR/suncong/NER_2021_github/output/BC4CHEMD_MRC_seed_17_top3_epochs_1_bert' is a path or url to a directory containing tokenizer files.
09/25/2021 20:46:27 - INFO - models.transformers.tokenization_utils -   Didn't find file /home/DUTIR/suncong/NER_2021_github/output/BC4CHEMD_MRC_seed_17_top3_epochs_1_bert/added_tok

[Predicting] 27120/27125 [============================>.] - ETA: 0ss0

09/25/2021 20:56:51 - INFO - root -   



[Predicting] 27125/27125 [==============================] 22.3ms/step

09/25/2021 20:56:52 - INFO - root -   

09/25/2021 20:56:52 - INFO - root -   right : 23155
09/25/2021 20:56:52 - INFO - root -   found : 24646
09/25/2021 20:56:52 - INFO - root -   origin : 25346
09/25/2021 20:56:52 - INFO - root -   right : 23155
09/25/2021 20:56:52 - INFO - root -   found : 24646
09/25/2021 20:56:52 - INFO - root -   origin : 25346
09/25/2021 20:56:52 - INFO - root -   ***** Test results  *****
09/25/2021 20:56:52 - INFO - root -    precision: 0.9395 - recall: 0.9136 - f1: 0.9263 - loss: 0.0068 
09/25/2021 20:56:52 - INFO - root -   ***** Test entity results  *****
09/25/2021 20:56:52 - INFO - root -   ******* BioNE results ********
09/25/2021 20:56:52 - INFO - root -    precision: 0.9395 - recall: 0.9136 - f1: 0.9263 


# 去除wordpiece影响，可以略微增强结果性能。

In [3]:
import json

with open(output_dir_url+"bert/test_predict.json", "r+") as f:
    data = f.readlines()


bert_pred_list = []
for i in range(len(data)):
    j = json.loads(data[i])
    token_seq = j['token_seq']
    tag_seq = j['tag_seq']

    bert_pred = dict({'toks':[], 'labels':[]})
    for t, l in zip(token_seq,tag_seq):
        if t in ['[SEP]']: # non-text tokens will not be evaluated.
            break
        elif t[:2] == '##': # if it is a piece of a word (broken by Word Piece tokenizer)
            bert_pred['toks'][-1] = bert_pred['toks'][-1]+t[2:] # append pieces
        else:
            bert_pred['toks'].append(t)
            bert_pred['labels'].append(l)
    bert_pred_list.append(bert_pred)

    

with open(output_dir_url+'bert/NER_result_conll.txt', 'w') as out_:
    for i in range(len(bert_pred_list)):
        bert_pred = bert_pred_list[i]
        idx=0
        for pred_t in bert_pred['toks']:
            out_.write("%s\t%s\n"%(bert_pred['toks'][idx], bert_pred['labels'][idx]))
            idx+=1
        out_.write("\n")
        
pre_label_list = []
with open(output_dir_url+'bert/NER_result_conll.txt') as a1:
    tmp_list = []
    for line in a1:
#         print(line)
        if line != '\n':
            token, label = line.split('\t')
            tmp_list.append(label[:-1])
        else:
            pre_label_list.append(tmp_list)
            tmp_list=[]
            
gs_label_list = []
with open('/home/DUTIR/suncong/NER_2021_github/datasets/BC4CHEMD/mrc-ner-top3-chem/test.txt') as a2:
    tmp_list = []
    for line in a2:
        if line != '\n':
            token, label = line.split('\t')
            tmp_list.append(label[:-1])
        else:
            gs_label_list.append(tmp_list)
            tmp_list=[]
            
from metric import get_ner_fmeasure
acc, P, R, F1 = get_ner_fmeasure(gs_label_list, pre_label_list, 'BIO')
print('P, R, F1:', P,  R,  F1)

output_test_file = os.path.join(args.output_dir, "test_eval_results.txt")
with open(output_test_file, "w") as writer:
    writer.write("P = {}\n".format(str(P)))
    writer.write("R = {}\n".format(str(R)))
    writer.write("F1 = {}\n".format(str(F1)))


gold_num =  25346  pred_num =  24645  right_num =  23157
P, R, F1: 0.939622641509434 0.9136352876193482 0.926446760416875
